<a id="start"></a>
# __School District Analysis__
### Justin Stubbs
[Conclusion](#end)

In [1]:
# Import libraries
import pandas as pd
import numpy as np

# Read in data
schools = pd.read_csv("Resources/schools_complete.csv")
students = pd.read_csv("Resources/students_complete.csv")

# Combine data into a single DataFrame
combined = schools.merge(students, on="school_name")
combined.head()

,School ID,school_name,type,size,budget,Student ID,student_name,gender,grade,reading_score,math_score
0,0,Huang High School,District,2917,1910635,0,Paul Bradley,M,9th,66,79
1,0,Huang High School,District,2917,1910635,1,Victor Smith,M,12th,94,61
2,0,Huang High School,District,2917,1910635,2,Kevin Rodriguez,M,12th,90,60
3,0,Huang High School,District,2917,1910635,3,Dr. Richard Scott,M,12th,67,58
4,0,Huang High School,District,2917,1910635,4,Bonnie Ray,F,9th,97,84


In [2]:
# Add columns for whether or not each student passed their math and reading tests
combined['pass_math'] = combined['math_score'] >= 70
combined['pass_reading'] = combined['reading_score'] >= 70
combined.head()

,School ID,school_name,type,size,budget,Student ID,student_name,gender,grade,reading_score,math_score,pass_math,pass_reading
0,0,Huang High School,District,2917,1910635,0,Paul Bradley,M,9th,66,79,True,False
1,0,Huang High School,District,2917,1910635,1,Victor Smith,M,12th,94,61,False,True
2,0,Huang High School,District,2917,1910635,2,Kevin Rodriguez,M,12th,90,60,False,True
3,0,Huang High School,District,2917,1910635,3,Dr. Richard Scott,M,12th,67,58,False,False
4,0,Huang High School,District,2917,1910635,4,Bonnie Ray,F,9th,97,84,True,True


In [3]:
# Calculate District statistics
tot_schools = schools['school_name'].count()
tot_students = students['student_name'].count()
tot_budget = schools['budget'].sum()
avg_math = students['math_score'].mean()
avg_reading = students['reading_score'].mean()
pass_math = combined['pass_math'].mean()
pass_reading = combined['pass_reading'].mean()
overall_pass = (pass_math + pass_reading) / 2

# Create new DataFrame using the above calulated stats for the District Summary
district_summary = pd.DataFrame({"Total Schools": tot_schools,
                        "Total Students": '{:,}'.format(tot_students),
                        "Total Budget": '$' + '{:,.2f}'.format(tot_budget),
                        "Average Math Score": '{:.3f}'.format(avg_math),
                        "Average Reading Score": '{:.3f}'.format(avg_reading),
                        "% Passing Math": '{:.3%}'.format(pass_math),
                        "% Passing Reading": '{:.3%}'.format(pass_reading),
                        "% Overall Passing Rate": ['{:.3%}'.format(overall_pass)]})
district_summary.head()

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,"39,170","$24,649,428.00",78.985,81.878,74.981%,85.805%,80.393%


In [4]:
# Group the combined student data by school
grouped_schools = combined.groupby(['school_name'], as_index=False)

# Calculate the average test scores and pass rates for each school,
# Save in a new DataFrame, dropping unnecessary columns
score_mean = grouped_schools.mean()
score_mean = score_mean.drop(columns=['School ID', 'size', 'budget', 'Student ID'])
score_mean.head()

,school_name,reading_score,math_score,pass_math,pass_reading
0,Bailey High School,81.033963,77.048432,0.666801,0.819333
1,Cabrera High School,83.975780,83.061895,0.941335,0.970398
2,Figueroa High School,81.158020,76.711767,0.659885,0.807392
3,Ford High School,80.746258,77.102592,0.683096,0.792990
4,Griffin High School,83.816757,83.351499,0.933924,0.971390


In [5]:
# Create a new DataFrame for the School Summary table
schools_summary = schools[['school_name', 'type', 'size', 'budget']]
schools_summary['Per Student Budget'] = schools['budget'] / schools['size']

# Merge the new School Summary table with the average scores and pass rates calculated above (on 'school_name')
schools_summary = schools_summary.merge(score_mean, on='school_name')
schools_summary['% Passing Overall'] = (schools_summary['pass_math'] + schools_summary['pass_reading']) / 2

# Rename columns for easy reading
schools_summary = schools_summary.rename(columns={'school_name': 'School', 'type': 'Type of School',
                                                  'size': 'Total Students', 'budget': 'Total Budget',
                                                  'reading_score': 'Average Reading Score',
                                                  'math_score': 'Average Math Score',
                                                  'pass_math': '% Passing Math', 'pass_reading': '% Passing Reading'})

# Sort school based on overall student pass rates
# Reset the indexing after sorting
schools_summary = schools_summary.sort_values('% Passing Overall', ascending=False).reset_index()
schools_summary = schools_summary.drop(columns='index')

schools_summary.head()

,School,Type of School,Total Students,Total Budget,Per Student Budget,Average Reading Score,Average Math Score,% Passing Math,% Passing Reading,% Passing Overall
0,Cabrera High School,Charter,1858,1081356,582.0,83.975780,83.061895,0.941335,0.970398,0.955867
1,Thomas High School,Charter,1635,1043130,638.0,83.848930,83.418349,0.932722,0.973089,0.952905
2,Pena High School,Charter,962,585858,609.0,84.044699,83.839917,0.945946,0.959459,0.952703
3,Griffin High School,Charter,1468,917500,625.0,83.816757,83.351499,0.933924,0.971390,0.952657
4,Wilson High School,Charter,2283,1319574,578.0,83.989488,83.274201,0.938677,0.965396,0.952037


In [6]:
# Save the top performing schools in a separate table
top_schools = schools_summary[:5]
top_schools

,School,Type of School,Total Students,Total Budget,Per Student Budget,Average Reading Score,Average Math Score,% Passing Math,% Passing Reading,% Passing Overall
0,Cabrera High School,Charter,1858,1081356,582.0,83.975780,83.061895,0.941335,0.970398,0.955867
1,Thomas High School,Charter,1635,1043130,638.0,83.848930,83.418349,0.932722,0.973089,0.952905
2,Pena High School,Charter,962,585858,609.0,84.044699,83.839917,0.945946,0.959459,0.952703
3,Griffin High School,Charter,1468,917500,625.0,83.816757,83.351499,0.933924,0.971390,0.952657
4,Wilson High School,Charter,2283,1319574,578.0,83.989488,83.274201,0.938677,0.965396,0.952037


In [7]:
# Save the worst performing schools in a separate table
bottom_schools = schools_summary[-5:].sort_values('% Passing Overall')
bottom_schools

,School,Type of School,Total Students,Total Budget,Per Student Budget,Average Reading Score,Average Math Score,% Passing Math,% Passing Reading,% Passing Overall
14,Rodriguez High School,District,3999,2547363,637.0,80.744686,76.842711,0.663666,0.802201,0.732933
13,Figueroa High School,District,2949,1884411,639.0,81.158020,76.711767,0.659885,0.807392,0.733639
12,Huang High School,District,2917,1910635,655.0,81.182722,76.629414,0.656839,0.813164,0.735002
11,Johnson High School,District,4761,3094650,650.0,80.966394,77.072464,0.660576,0.812224,0.736400
10,Ford High School,District,2739,1763916,644.0,80.746258,77.102592,0.683096,0.792990,0.738043


In [8]:
# Show the average math test score for each grade level at each school
school_grade_groups = combined.groupby(['school_name', 'grade'])
school_grade_math = pd.DataFrame(school_grade_groups['math_score'].mean())
school_grade_math = school_grade_math.rename(columns={'math_score': 'Average Math Score'}).unstack(level=1)
school_grade_math

Average Math Score                                 
grade                               10th       11th       12th        9th
school_name                                                              
Bailey High School             76.996772  77.515588  76.492218  77.083676
Cabrera High School            83.154506  82.765560  83.277487  83.094697
Figueroa High School           76.539974  76.884344  77.151369  76.403037
Ford High School               77.672316  76.918058  76.179963  77.361345
Griffin High School            84.229064  83.842105  83.356164  82.044010
Hernandez High School          77.337408  77.136029  77.186567  77.438495
Holden High School             83.429825  85.000000  82.855422  83.787402
Huang High School              75.908735  76.446602  77.225641  77.027251
Johnson High School            76.691117  77.491653  76.863248  77.187857
Pena High School               83.372000  84.328125  84.121547  83.625455
Rodriguez High School          76.612500  76.395626  77.690748  76.859966
Shelton High School            82.917411  83.383495  83.778976  83.420755
Thomas High School             83.087886  83.498795  83.497041  83.590022
Wilson High School             83.724422  83.195326  83.035794  83.085578
Wright High School             84.010288  83.836782  83.644986  83.264706

In [9]:
# Show the average reading test score for each grade level at each school
school_grade_reading = pd.DataFrame(school_grade_groups['reading_score'].mean())
school_grade_reading = school_grade_reading.rename(columns={'reading_score': 'Average Reading Score'}).unstack(level=1)
school_grade_reading

Average Reading Score                                 
grade                                  10th       11th       12th        9th
school_name                                                                 
Bailey High School                80.907183  80.945643  80.912451  81.303155
Cabrera High School               84.253219  83.788382  84.287958  83.676136
Figueroa High School              81.408912  80.640339  81.384863  81.198598
Ford High School                  81.262712  80.403642  80.662338  80.632653
Griffin High School               83.706897  84.288089  84.013699  83.369193
Hernandez High School             80.660147  81.396140  80.857143  80.866860
Holden High School                83.324561  83.815534  84.698795  83.677165
Huang High School                 81.512386  81.417476  80.305983  81.290284
Johnson High School               80.773431  80.616027  81.227564  81.260714
Pena High School                  83.612000  84.335938  84.591160  83.807273
Rodriguez High School             80.629808  80.864811  80.376426  80.993127
Shelton High School               83.441964  84.373786  82.781671  84.122642
Thomas High School                84.254157  83.585542  83.831361  83.728850
Wilson High School                84.021452  83.764608  84.317673  83.939778
Wright High School                83.812757  84.156322  84.073171  83.833333

In [10]:
# Show average school performance based on budget for each student
spending_bins = [0, 585, 615, 645, 675]
spending_group_names = ["<$585", "$585-615", "$615-645", "$645-675"]
schools_summary['Spending Per Student'] = pd.cut(schools_summary['Per Student Budget'], spending_bins, labels=spending_group_names)
spending_group = schools_summary.groupby(['Spending Per Student']).mean()
spending_group = spending_group[['Average Math Score', 'Average Reading Score', '% Passing Math',
                                '% Passing Reading', '% Passing Overall']]
spending_group

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
Spending Per Student,,,,,
<$585,83.455399,83.933814,0.934601,0.966109,0.950355
$585-615,83.599686,83.885211,0.942309,0.959003,0.950656
$615-645,79.079225,81.891436,0.756682,0.861066,0.808874
$645-675,76.997210,81.027843,0.661648,0.811340,0.736494


In [11]:
# Show average school performance based on size of the school
size_bins = [0, 1000, 2000, 5000]
size_group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]
schools_summary['School Size'] = pd.cut(schools_summary['Total Students'], size_bins, labels=size_group_names)
size_group = schools_summary.groupby(['School Size']).mean()
size_group = size_group[['Average Math Score', 'Average Reading Score', '% Passing Math',
                                '% Passing Reading', '% Passing Overall']]
size_group

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
School Size,,,,,
Small (<1000),83.821598,83.929843,0.935502,0.960994,0.948248
Medium (1000-2000),83.374684,83.864438,0.935997,0.967907,0.951952
Large (2000-5000),77.746417,81.344493,0.699634,0.827666,0.763650


In [12]:
# Show average school performance based on type of school
school_types = schools_summary.groupby(['Type of School'])
school_types = school_types[['Average Math Score', 'Average Reading Score', '% Passing Math',
                                '% Passing Reading', '% Passing Overall']].mean()
school_types

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
Type of School,,,,,
Charter,83.473852,83.896421,0.936208,0.965865,0.951037
District,76.956733,80.966636,0.665485,0.807991,0.736738


<a id="end"></a>
# __Results and Conclusion__

It is clear from the data that Charter schools far out-perform District schools. This is evidenced by the 22-percentage point difference in the average overall performance of students in Charter schools vs District schools. In this light, it should be no surprise that the top 5 performing schools in the District are all Charter schools.

Additionally, school size seems to play a larger role on student passing rates than per-student-spending - smaller schools tended to out-perform schools with larger budgets. This could be due to lower student to teacher ratios in smaller schools, however this is unconfirmed due to lack of data. This also shows that throwing money at the problem does not always achieve the deisred results.

[Return to Top](#start)